Notebook to slice maxar images like drones, and to generate training samples as pairs maxar-planet

In [1]:
import os
import shutil
import json
import subprocess
from datetime import datetime, timedelta

from tqdm import tqdm
import numpy as np
import pandas as pd
import rasterio
import rasterio.mask
from shapely.geometry import box, mapping, shape
from rasterio.plot import reshape_as_image, reshape_as_raster
from rasterio.features import shapes
import geopandas as gpd
import cv2
from skimage import exposure

import os
from dataclasses import dataclass
import shutil
import re

from tqdm import tqdm
import numpy as np
import geopandas as gpd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import rasterio
from rasterio.windows import Window
from rasterio.enums import Resampling
from rasterio import features as rasterio_features
from affine import Affine
from rasterio.plot import reshape_as_image, reshape_as_raster
from shapely.geometry import box
from rasterio.features import geometry_mask
from shapely.geometry import Polygon, MultiPolygon, shape
import rasterio.mask


from sentinel2download.downloader import Sentinel2Downloader
from sentinel2download.overlap import Sentinel2Overlap

# from utils import crop_raster, merge_tiles, stitch_tiles, get_tiles

In [2]:
maxar_images_root = '/home/mlatushkina/satelite_nav/maxar/sip_landcover/test/maxar/images'

maxar_aoi_root = '/home/mlatushkina/satelite_nav/maxar/sip_landcover/test/maxar/aoi'
basemap_dir = '/home/mlatushkina/satelite_nav/maxar/sip_landcover/test/maxar/sentinel'

maxar_drones_dir = '/home/mlatushkina/satelite_nav/maxar/sip_landcover/test/maxar/maxar_tiles'

train_data_root = '/home/mlatushkina/satelite_nav/maxar/sip_landcover/test/maxar/train'

In [3]:
# shutil.rmtree(maxar_aoi_root, ignore_errors=True)
# os.mkdir(maxar_aoi_root)

# shutil.rmtree(basemap_dir, ignore_errors=True)
# os.mkdir(basemap_dir)

# shutil.rmtree(maxar_drones_dir, ignore_errors=True)
# os.mkdir(maxar_drones_dir)

# shutil.rmtree(train_data_root, ignore_errors=True)
# os.mkdir(train_data_root)

In [4]:
maxar_data = '/home/mlatushkina/satelite_nav/maxar/sip_landcover/turkey_maxar.csv'

In [5]:
maxar_df = pd.read_csv(maxar_data)
maxar_df.head()

,link,name,date,group
0,https://maxar-opendata.s3.us-west-2.amazonaws....,103001005A587700-visual.tif,2016-07-28,31133012131
1,https://maxar-opendata.s3.us-west-2.amazonaws....,104001002AA16300-visual.tif,2017-02-15,31133012101
2,https://maxar-opendata.s3.us-west-2.amazonaws....,104001002AA16300-visual.tif,2017-02-15,31133010323
3,https://maxar-opendata.s3.us-west-2.amazonaws....,104001002AA16300-visual.tif,2017-02-15,31133010322
4,https://maxar-opendata.s3.us-west-2.amazonaws....,104001002AA16300-visual.tif,2017-02-15,31133010321


In [6]:
import pandas as pd
import urllib.request

data = pd.read_csv('./turkey_maxar.csv')
data['year'] = data['date'].str.split('-', expand=True)[0]
data = data[data['year'] == '2023']
testfile = urllib.request.URLopener()

for index, row in data.iterrows():
    print(index)
    testfile.retrieve(row['link'], "/home/mlatushkina/satelite_nav/maxar/sip_landcover/test/maxar/images/" + str(row['date']) + '_' + str(row['group']) + '_' + str(row['name']))

325
326


KeyboardInterrupt: 

In [ ]:
# maxar_df = pd.read_json(maxar_data, orient='records', dtype={'month': str, 'year': str, 'image_id': str})
# maxar_df['name'] = maxar_df['image_id'] + '-visual.tif'
# maxar_df.head()


## Step 1: Load planet basemap for maxar

repo for downloadingPlanet basemaps: https://github.com/QuantuMobileSoftware/planet_basemap_downloader

Create AOI from maxar rasters

In [7]:
maxar_df = pd.read_csv('./turkey_maxar.csv')

In [8]:
maxar_df['month'] = maxar_df['date'].apply(lambda x: x.split('-')[1])
maxar_df['year'] = maxar_df['date'].apply(lambda x: x.split('-')[0])
maxar_df['image_id'] = maxar_df['name'].apply(lambda x: x.split('-')[0])


In [9]:
maxar_df = maxar_df[maxar_df['year'] == '2023']

In [11]:
def save_image_polygon(im_path, save_path):
    with rasterio.open(im_path) as src:
        bounds = src.bounds
        bbox = box(bounds.left, bounds.bottom, bounds.right, bounds.top)
        gdf = gpd.GeoDataFrame(geometry=[bbox], crs=src.crs)
        gdf.to_file(save_path, driver='GeoJSON')

In [12]:
saved = os.listdir(maxar_images_root)
saved[0]

'2023-01-07_31133031301_10400100805A3300-visual.tif'

In [ ]:
for _, row in maxar_df.iterrows():
    name = str(row['date']) + '_' + str(row['group']) + '_' + str(row['name'])
    maxar_path = os.path.join(maxar_images_root, name)
    if name in saved:
        maxar_aoi_path = os.path.join(maxar_aoi_root, row['image_id'] + '-aoi.geojson')
        save_image_polygon(maxar_path, maxar_aoi_path)


run planet_basemap_downloader

In [13]:
maxar_df

,link,name,date,group,month,year,image_id
325,https://maxar-opendata.s3.us-west-2.amazonaws....,10300100DFA9DD00-visual.tif,2023-01-02,31133113121,01,2023,10300100DFA9DD00
326,https://maxar-opendata.s3.us-west-2.amazonaws....,10300100DFA9DD00-visual.tif,2023-01-02,31133113120,01,2023,10300100DFA9DD00
327,https://maxar-opendata.s3.us-west-2.amazonaws....,10300100DF1DB000-visual.tif,2023-01-02,31133102211,01,2023,10300100DF1DB000
328,https://maxar-opendata.s3.us-west-2.amazonaws....,10300100DFA9DD00-visual.tif,2023-01-02,31133113100,01,2023,10300100DFA9DD00
329,https://maxar-opendata.s3.us-west-2.amazonaws....,10300100DFA9DD00-visual.tif,2023-01-02,31133113031,01,2023,10300100DFA9DD00
...,...,...,...,...,...,...,...
1575,https://maxar-opendata.s3.us-west-2.amazonaws....,10300100E32E3D00-visual.tif,2023-02-28,31133032021,02,2023,10300100E32E3D00
1576,https://maxar-opendata.s3.us-west-2.amazonaws....,10300100E32E3D00-visual.tif,2023-02-28,31133032022,02,2023,10300100E32E3D00
1577,https://maxar-opendata.s3.us-west-2.amazonaws....,10300100E32E3D00-visual.tif,2023-02-28,31133032023,02,2023,10300100E32E3D00
1578,https://maxar-opendata.s3.us-west-2.amazonaws....,10300100E32E3D00-visual.tif,2023-02-28,31133023113,02,2023,10300100E32E3D00


In [15]:
def shift_date(date, delta=5, format='%Y-%m-%d'):
    date = datetime.strptime(date, format)
    date = date - timedelta(days=delta)    
    return datetime.strftime(date, format)


def load_images(tiles, start_date, end_date, output_dir):
    loader = Sentinel2Downloader(SENTINEL2_GOOGLE_API_KEY)
    
    loadings = dict()
        
    for tile in tiles:
        start = start_date
        end = end_date
        
        print(f"Loading images for tile: {tile}...")
        count = 0
        while count < MAX_SHIFT_ITERS:
            loaded = loader.download(PRODUCT_TYPE,
                                [tile],
                                start_date=start,
                                end_date=end,
                                output_dir=output_dir,                       
                                bands=BANDS,
                                constraints=CONSTRAINTS)
        
            if not loaded:
                end = start_date
                start = shift_date(start_date, delta=MAX_SHIFT) 
                print(f"For tile: {tile} and dates {start_date} {end_date} proper images not found! Shift dates to {start} {end}!")
            else:
                break
            count += 1
        if loaded:
            loadings[tile] = loaded
            print(f"Loading images for tile {tile} finished")
        else:
            print(f"Images for tile {tile} were not loaded!")
        
    tile_folders = dict()
    for tile, tile_paths in loadings.items():
        tile_folders[tile] = str(Path(tile_paths[0][0]).parent)#{str(Path(tile_path[0]).parent) for tile_path in tile_paths}
    return loadings, tile_folders

In [16]:
from pathlib import Path

In [20]:
START_DATE = os.getenv('START_DATE', '2023-05-01')
END_DATE = os.getenv('END_DATE', '2023-06-30')

SENTINEL2_GOOGLE_API_KEY = './google_api_key.json'
MAX_SHIFT_ITERS = 2
MAX_SHIFT = 30
PRODUCT_TYPE = 'L2A'
BANDS = {"B02", "B03", "B04"}
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 10.0, 'CLOUDY_PIXEL_PERCENTAGE': 5.0, }

loaded = []
# tiles = set()
for _, row in maxar_df.iterrows():
    aoi = os.path.join(maxar_aoi_root, row['image_id'] + '-aoi.geojson')
    aoi_filename = os.path.join(maxar_aoi_root, aoi)
    if os.path.exists(aoi_filename):
        aoi = gpd.read_file(aoi_filename)
        aoi = aoi.to_crs('EPSG:4326')
        polygon = aoi.loc[0, 'geometry']
        gpd.GeoDataFrame(
            geometry=[polygon]).to_file(aoi_filename, driver="GeoJSON", crs='EPSG:4326')
        s2overlap = Sentinel2Overlap(aoi_path=aoi_filename)

        basemap_path = os.path.join(basemap_dir, row['image_id'])
    #     shutil.rmtree(basemap_path, ignore_errors=True)
    #     os.mkdir(basemap_path)
        # overlap_tiles = set(s2_grid[s2_grid.intersects(polygon)].Name.apply(lambda x:x[:3]).to_list())

        start_year = int(START_DATE.split("-")[0])
        end_year = int(END_DATE.split("-")[0])

        overlap_tiles = s2overlap.overlap()
        if overlap_tiles[0] not in tiles:
            tiles.add(overlap_tiles[0])
            loadings, folder = load_images(overlap_tiles, START_DATE, END_DATE, basemap_dir)
            print(folder)
            print(overlap_tiles[0])
            os.rename(folder[overlap_tiles[0]], basemap_path)
            loaded.append(loadings)
            print(aoi)

In [ ]:
from shutil import move

In [19]:
tiles

{'36SYF', '36SYG', '37SBB', '37SCA', '37SCB', '37SDB'}

In [ ]:
def get_rgb(bands, output):
    band_red = rasterio.open(bands[0])
    band_green = rasterio.open(bands[1])
    band_blue = rasterio.open(bands[2])

    red = band_red.read(1).astype('float32')
    green = band_green.read(1).astype('float32')
    blue = band_blue.read(1).astype('float32')

    red /= 10000.0
    green /= 10000.0
    blue /= 10000.0

    red = np.clip(red, 0, 1)
    green = np.clip(green, 0, 1)
    blue = np.clip(blue, 0, 1)

    profile = band_red.profile
    profile.update(count=3, dtype='uint8', driver='GTiff')

    with rasterio.open(output, 'w', **profile) as dst:
        rgb = np.stack((red, green, blue), axis=0)
        rgb = (rgb * 255).astype('uint8')
        dst.write(rgb)

    band_red.close()
    band_green.close()
    band_blue.close()
    for i in bands:
        os.remove(i)


In [ ]:
for folder in os.listdir(basemap_dir):
    tile_folder = os.path.join(basemap_dir, folder)
    bands = [os.path.join(tile_folder, file) for file in os.listdir(tile_folder) if file.endswith('.jp2')]
    if len(bands) == 3:
        bands.sort(reverse=True)
        get_rgb(bands, os.path.join(tile_folder, f'{folder}.tif'))
        for file in os.listdir(tile_folder):
            if not file.endswith('tif'):
                print(os.path.join(tile_folder, file))
                os.remove(os.path.join(tile_folder, file))
    else:
        shutil.rmtree(tile_folder)

## Step 2: slice maxar

In [ ]:
def get_image_polygon_df(im_path, buffer_mult=1):
    with rasterio.open(im_path) as src:
        bbox = box(*src.bounds)
        if buffer_mult > 1:
            bbox_size = np.sqrt(bbox.area)
            bbox = bbox.buffer((buffer_mult-1)*bbox_size, cap_style=2)
        gdf = gpd.GeoDataFrame(geometry=[bbox], crs=src.crs)
        return gdf

def get_raster_crs(raster_path):
    with rasterio.open(raster_path) as src:
        return src.crs


def crop_raster(polygon, src_path, dst_path, to_rgb=False):
    """
    Crop a Sentinel 2 raster using a Shapely Polygon object.

    Args:
        polygon (shapely.geometry.Polygon): The polygon object used for cropping.
        src_path (str): The path to the source raster file.
        dst_path (str): The path to the destination cropped raster file.

    Returns:
        None
    """
    with rasterio.open(src_path) as src:

        out_image, out_transform = rasterio.mask.mask(src, [polygon], crop=True, nodata=0)
        print(out_image.shape)
        if to_rgb:
            out_image = reshape_as_image(out_image)
            out_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB)
            out_image = reshape_as_raster(out_image)
        print(out_image.shape)
        out_image = out_image[:3, :, :]
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                "count": 3,
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
        
        with rasterio.open(dst_path, "w", **out_meta) as dest:
            dest.write(out_image)

In [ ]:
from typing import List
from shapely.geometry import Polygon
from random import uniform
import math

def get_bounds_vector(min_coord, max_coord, min_len, max_len):
    row_start = row_end = min_coord
    rows_bounds = []
    while row_end < max_coord:
        row_start = row_end
        row_end = row_start + np.random.randint(min_len, max_len)
        rows_bounds.append((row_start, row_end))
    return rows_bounds


def make_mosaic(polygon: Polygon, min_size: float=200, max_size: float=1000) -> List[Polygon]:
    """
    Generates random rectangular polygons within the given polygon, with sizes ranging from min_size to max_size in meters.

    Args:
        polygon (Polygon): The input polygon in which to generate rectangular mosaics.
        min_size (float, optional): The minimum size (in meters) of the mosaics. Defaults to 200.
        max_size (float, optional): The maximum size (in meters) of the mosaics. Defaults to 1000.

    Returns:
        List[Polygon]: A list of generated mosaics as Shapely Polygon objects.
    """
    min_size = max(min_size, 0)
    max_size = max(max_size, min_size)

    minx, miny, maxx, maxy = polygon.bounds

    rows_bounds = get_bounds_vector(miny, maxy, min_size, max_size)

    tiles = []
    metadata = []
    for row_bound in rows_bounds:
        H = row_bound[1] - row_bound[0]
        min_col_size = max(min_size, int(0.5*H))
        max_col_size = max(max_size, int(1.5*H))
        col_bounds = get_bounds_vector(minx, maxx, min_col_size, max_col_size)
        for col_bound in col_bounds:
            x1, x2 = col_bound
            y1, y2 = row_bound
            tile = Polygon([(x1, y1), (x1, y2), (x2, y2), (x2, y1), (x1, y1)])

            if tile.within(polygon):
                tiles.append(tile)
                metadata.append({'x0': x1, 'x1': x2, 'y0': y1, 'y1': y2})
    tiles_df = gpd.GeoDataFrame(metadata, geometry=tiles)
    return tiles_df

def slice_tif_on_tiles(tif_path, tiles_dest_dir, min_size: float=200, max_size: float=1000, to_rgb=False):
    poly_df = get_image_polygon_df(tif_path)
    raster_crs = get_raster_crs(tif_path)
    poly_df.to_crs(raster_crs, inplace=True)
    poly = poly_df['geometry'].iloc[0]

    tiles_df = make_mosaic(poly, min_size, max_size)
    tiles_df.set_crs(raster_crs)  

    for tile_id, tile_meta in tqdm(tiles_df.iterrows(), total=len(tiles_df)):
        tile_name = f"tile_{tile_id}.tif"
        tile_path = os.path.join(tiles_dest_dir, tile_name)
        crop_raster(tile_meta['geometry'], tif_path, tile_path, to_rgb)


In [ ]:
shutil.rmtree(maxar_drones_dir, ignore_errors=True)
os.mkdir(maxar_drones_dir)

In [ ]:
maxar_df

In [ ]:
for i, row in maxar_df.iterrows():
    print(i)
    maxar_path = os.path.join(maxar_images_root, str(row['date']) + '_' + str(row['group']) + '_' + str(row['name']))
    drones_subfolder = os.path.join(maxar_drones_dir, row['image_id'])

    shutil.rmtree(drones_subfolder, ignore_errors=True)
    os.mkdir(drones_subfolder)

    slice_tif_on_tiles(maxar_path, drones_subfolder, 750*3, 850*3, to_rgb=False)


In [ ]:
drones_subfolder

## Step 3: Create training pairs

Creating train pairs as:
- [satellite id]-[tile id]_UAV.tif - drone image
- [satellite id]-[tile id].tif - satellite image

For PlanetScope basemap, satellite id is the name as maxar raster

In [ ]:
def pad_raster(src_path, dst_path, desired_coverage=(720,960), min_pad_ratio=1.2):
    """
    Pad raster by coverage in meters.

    Args:
        src_path (str): The path to the source raster file.
        dst_path (str): The path to the destination of padded raster file.
        desired_coverage (Tuple[float]): size in meters of output raster
        min_pad_ratio (float): minimal (out size)/(input size). 
            If padding size ratio estimated from desired_coverage is less then min_pad_ratio, 
            min_pad_ratio will be used.
    """
    with rasterio.open(src_path) as src:
        bounds = src.bounds

        center_x = (bounds.left + bounds.right) / 2
        center_y = (bounds.bottom + bounds.top) / 2

        cover_now = (int(bounds.right - bounds.left), int(bounds.top - bounds.bottom))
        
        if cover_now[0] < cover_now[1]:
            desired_coverage = (min(desired_coverage), max(desired_coverage))
        else:
            desired_coverage = (max(desired_coverage), min(desired_coverage))
        
        pad_size = max(desired_coverage[0]/cover_now[0], desired_coverage[1]/cover_now[1], min_pad_ratio)        
        scale_x, scale_y = pad_size, pad_size

        out_image = src.read()
        out_meta = src.meta
        out_transform = src.transform
        
        translate_to_origin = Affine.translation(-center_x, -center_y)
        scaling = Affine.scale(scale_x, scale_y)
        translate_back = Affine.translation(center_x, center_y)

        result_affine = translate_back * scaling * translate_to_origin
        out_transform = result_affine * out_transform

        pad1 = int((int(src.width * result_affine.a) - src.width)/2)
        pad2 = int((int(src.height * result_affine.e) - src.height)/2)
        out_image = np.pad(out_image, ((0, 0), (pad2, pad2), (pad1, pad1)), 'constant', constant_values=0)


        out_meta.update({"driver": "GTiff",
                 "transform": out_transform
                 })

        with rasterio.open(dst_path, "w", **out_meta) as dest:
            dest.write(out_image)

        with rasterio.open(dst_path) as src:
            bounds = src.bounds
            actual_coverage = (bounds.right-bounds.left, bounds.top-bounds.bottom)
        return actual_coverage

In [ ]:
shutil.rmtree(train_data_root, ignore_errors=True)
os.mkdir(train_data_root)

In [ ]:
basemap_dir = '/home/mlatushkina/satelite_nav/maxar/sip_landcover/test/maxar/sentinel'
# basemap_dir = '/home/mlatushkina/satelite_nav/maxar/sip_landcover/test/maxar/planet'

In [ ]:
for _, row in maxar_df.iterrows():
    # get path to subfolder where tiles are stored, and names of those tiles
    drones_subfolder = os.path.join(maxar_drones_dir, row['image_id'])
    drones_images_names = os.listdir(drones_subfolder)

    # path where we saved basemap tif in step 1
    basemap_path = os.path.join(basemap_dir, row['image_id'], row['image_id']+'.tif')
#     basemap_path = '/home/mlatushkina/satelite_nav/maxar/sip_landcover/output_image.tif'

    # iterate over maxar crops (like they are drone images)
    for drone_img_name in tqdm(drones_images_names, desc=f"Processing {row['image_id']}"):
        drone_photo_path = os.path.join(drones_subfolder, drone_img_name)

        # create path where to store padded drone image
        # Example: tile "tile_22.tif", image_id "103001003C3E4500", sample name "103001003C3E4500-tile-22_UAV.tif"
        drone_suffix = drone_img_name.replace('.tif', '').replace('_', '-')
        drone_padded_photo_name = f"{row['image_id']}-{drone_suffix}_UAV.tif"
        drone_padded_photo_path = os.path.join(train_data_root, drone_padded_photo_name)

        # desired coverage in meters. We want to get ~ 240 by 320 pixels in the basemap crop. 
        # The basemap has a resolution of 3.8. Therefore, coverage is 240*3.8, 320*3.8.
        desired_coverage = (912, 1216)
        
        pad_raster(drone_photo_path, drone_padded_photo_path, desired_coverage, min_pad_ratio=1.01)

        # we read padded drone image to crop satellite to the same aoi
        padded_image_bounds = get_image_polygon_df(drone_padded_photo_path)
        sat_crs = get_raster_crs(basemap_path)
        padded_image_bounds.to_crs(sat_crs, inplace=True)
        padded_image_bounds = padded_image_bounds['geometry'].iloc[0]

        # cropped satellite name should be the same as corresponding UAV image, but without _UAV suffix
        sat_cropped_photo_path = drone_padded_photo_path.replace('_UAV.tif', '.tif')
        crop_raster(padded_image_bounds, basemap_path, sat_cropped_photo_path)


# plot single

In [ ]:
with rasterio.open(drone_padded_photo_path) as src:
    drone_image = reshape_as_image( src.read())
    
with rasterio.open(sat_cropped_photo_path) as src:
    sat_image = reshape_as_image( src.read())

drone_image.shape, sat_image.shape

In [ ]:
drone_image = cv2.resize(drone_image, sat_image.shape[:2][::-1])
drone_image.shape

In [ ]:
plt.figure(figsize=(16,16))

comb = drone_image.copy()
slice_x = 110
comb[:, :slice_x, :] = sat_image[:, :slice_x, :]
plt.imshow(comb)
# axs[1].imshow(drone_image, alpha=0.2)


In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(16,16))
axs = axs.flatten()
axs[0].imshow(sat_image)
axs[1].imshow(drone_image)


axs[2].imshow(sat_image)
axs[2].imshow(drone_image, alpha=0.4)

comb = drone_image.copy()
slice_x = 220
comb[:, :slice_x, :] = sat_image[:, :slice_x, :]
axs[3].imshow(comb)
